## 1. 환경 설정

`(1) Env 환경변수`

In [13]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 라이브러리`

In [14]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

## 2. 도구 호출 (Tool Calling)
- 도구 호출은 LLM이 특정 작업을 수행하기 위해 외부 기능을 호출하는 기능
- 이를 통해 LLM은 외부 API 통합 등 더 복잡한 작업을 수행할 수 있음 

### 2-1. 랭체인 내장 도구
- Tavily 웹 검색 도구 (예시)

`(1) 도구(tool) 정의하기`

In [ ]:
from langchain_community.tools import TavilySearchResults

# 검색할 쿼리 설정
query = "스테이크와 어울리는 와인을 추천해주세요."

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
# 웹 검색
# max_result - 최대 검색 개수
web_search = TavilySearchResults(max_results=12)

# 웹 검색 실행
search_results = web_search.invoke(query)

# 검색 결과 출력
for result in search_results:
    print(result)  
    print("-" * 100)  

{'title': '스테이크와 찰떡궁합! 스테이크에 어울리는 와인 5가지 - 술의 모든 것', 'url': 'https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80', 'content': '• 2. 시라/쉬라즈 (Syrah/Shiraz) – 스파이시한 풍미가 매력적인 와인\n\n• 3. 말벡 (Malbec) – 풍부한 과일 향과 부드러운 탄닌\n\n• 4. 메를로 (Merlot) – 부드럽고 균형 잡힌 와인\n\n• 5. 진판델 (Zinfandel) – 달콤한 향과 강한 바디감\n\n• 어떤 와인을 선택해야 할까?\n\n스테이크를 제대로 즐기려면 와인 선택이 중요합니다. 고기의 종류와 조리법에 따라 어울리는 와인이 달라지는데요. 어떤 와인을 선택해야 풍미를 극대화할 수 있을까요? 오늘은 스테이크와 가장 잘 어울리는 와인 5가지를 소개합니다.\n\n  \n\n1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\n------------------------------------------------\n\nImage 3Image 4\n\n추천 이유: [...] 카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\n\n어울리는 스테이크:\n\n   리브아이 스테이크\n   뉴욕 스트립\n   티본 스테이크\n\n추천 와인:\n\n   나파 밸리 카베르네 소비뇽\n   샤또 마고 (Château Margaux)\n\n카베르네 소비뇽은 특히 마블링이 많은 소고기 스테

In [ ]:
# 도구 속성
print("자료형: ")
print(type(web_search))
print("-"*100)

print("name: ")
print(web_search.name)
print("-"*100)

print("description: ")
pprint(web_search.description)
print("-"*100)

# 한글 질문을 내부에서 자동으로 영어로 번역을 통해 검색하는 것으로 보임 .. ? 
print("schema: ")
pprint(web_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
----------------------------------------------------------------------------------------------------
name: 
tavily_search_results_json
----------------------------------------------------------------------------------------------------
description: 
('A search engine optimized for comprehensive, accurate, and trusted results. '
 'Useful for when you need to answer questions about current events. Input '
 'should be a search query.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input for the Tavily tool.',
 'properties': {'query': {'description': 'search query to look up',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'TavilyInput',
 'type': 'object'}
----------------------------------------------------------------------------------------------------

`(2) 도구(tool) 호출하기`

In [60]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# 웹 검색 도구를 직접 LLM에 바인딩 가능
# 검색 결과를 결합 ? 함 
llm_with_tools = llm.bind_tools(tools=[web_search])
print(type(llm))
print(type(llm_with_tools))

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
<class 'langchain_core.runnables.base.RunnableBinding'>


In [61]:
# 도구 호출이 필요 없는 LLM 호출을 수행
query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 87, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BoQVX8UKbl1a0La7d5XH6ghEShlae', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ec2e24f9-de02-4bb2-91d6-fcd8eab8b000-0', usage_metadata={'input_tokens': 87, 'output_tokens': 22, 'total_tokens': 109, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})
----------------------------------------------------------------------------------------------------
'안녕하세요! 무엇을 도와드릴까요?'
-------------------------------------------------------------------------

In [62]:
from pprint import pprint

# 도구 호출이 필요한 LLM 호출을 수행
query = "스테이크와 어울리는 와인을 추천해주세요."
#query = "KL Grade 3 등급, 골극의 갯수가 2개, JSW 감소율이 정상대비 15%이고 48 개월 이내에 oa progression이 진행할 가능성이 존재하는 환자야 해당 내용들을 통해서 종합적인 진단을 해줘"
ai_msg = llm_with_tools.invoke(query)

# AIMessage의 속성 확인
pprint(vars(ai_msg))


{'additional_kwargs': {'refusal': None,
                       'tool_calls': [{'function': {'arguments': '{"query":"wines '
                                                                 'that pair '
                                                                 'well with '
                                                                 'steak"}',
                                                    'name': 'tavily_search_results_json'},
                                       'id': 'call_3LsmqtbHbUruyAHx625hicwl',
                                       'type': 'function'}]},
 'content': '',
 'example': False,
 'id': 'run--c1e9efc2-9a45-42c1-8655-e8aa9e3fa29b-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'id': 'chatcmpl-BoQVZpmwJ6OYhKKCJ6e52ffEvOPmx',
                       'logprobs': None,
                       'model_name': 'gpt-3.5-turbo-0125',
                       'service_tier': 'default',
       

In [63]:

pprint(ai_msg)
print("#" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("*" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3LsmqtbHbUruyAHx625hicwl', 'function': {'arguments': '{"query":"wines that pair well with steak"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 103, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BoQVZpmwJ6OYhKKCJ6e52ffEvOPmx', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c1e9efc2-9a45-42c1-8655-e8aa9e3fa29b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'wines that pair well with steak'}, 'id': 'call_3LsmqtbHbUruyAHx625hicwl', 'type': 'tool_call'}], usage_metadata={'input_tokens': 103, 

In [65]:
tool_call = ai_msg.tool_calls[0]

print(tool_call)

{'name': 'tavily_search_results_json', 'args': {'query': 'wines that pair well with steak'}, 'id': 'call_3LsmqtbHbUruyAHx625hicwl', 'type': 'tool_call'}


`(3) 도구(tool) 실행하기`

#### 방법 1: 직접 도구 호출 처리한 결과로 ToolMessage 객체 생성

* 이 방법은 AI 메시지에서 첫 번째 도구 호출을 가져와 직접 처리한다.
* 'args'를 사용하여 도구를 호출하고 결과를 얻는다.
* 도구 호출 결과를 사용하여 ToolMessage 객체를 생성한다.
* 도구 호출의 ID와 이름을 포함하여 더 구조화된 ToolMessage 메시지를 만든다.

In [66]:
tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출 결과:")
print("-" * 100)

pprint(tool_output)

tavily_search_results_json 호출 결과:
----------------------------------------------------------------------------------------------------
[{'content': 'While red wine is often the traditional choice for pairing with '
             'steak, white wine can also be a good option. The key is to '
             'choose a white wine that’s full-bodied and rich enough to stand '
             'up to the flavor of the steak. Some good options for white wine '
             'pairings with steak include Chardonnay, Sauvignon Blanc, and '
             'Pinot Grigio. [...] The type of steak can have a significant '
             'impact on the wine pairing. Different types of steak have '
             'different flavor profiles, and some wines are better suited to '
             'certain types of steak than others. For example, a rich, fatty '
             'steak like a ribeye or a porterhouse pairs well with a '
             'full-bodied wine like a Cabernet Sauvignon or a Syrah/Shiraz. '
             'T

In [67]:

from langchain_core.messages import ToolMessage

tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call['id'],
    name=tool_call["name"]
)

pprint(tool_message.content)

[{'content': 'While red wine is often the traditional choice for pairing with '
             'steak, white wine can also be a good option. The key is to '
             'choose a white wine that’s full-bodied and rich enough to stand '
             'up to the flavor of the steak. Some good options for white wine '
             'pairings with steak include Chardonnay, Sauvignon Blanc, and '
             'Pinot Grigio. [...] The type of steak can have a significant '
             'impact on the wine pairing. Different types of steak have '
             'different flavor profiles, and some wines are better suited to '
             'certain types of steak than others. For example, a rich, fatty '
             'steak like a ribeye or a porterhouse pairs well with a '
             'full-bodied wine like a Cabernet Sauvignon or a Syrah/Shiraz. '
             'These wines have a rich, fruity flavor and a smooth, velvety '
             'texture that complements the fatty flavor of the steak. [..

#### 방법 2: 도구 직접 호출하여 바로 ToolMessage 객체 생성

* 이 방법은 도구를 직접 호출하여 ToolMessage 객체를 생성한다.
* 가장 간단하고 직관적인 방법으로, LangChain의 추상화를 활용한다.

In [68]:
# tool_call  {'name': 'tavily_search_results_json', 'args': {'query': 'wine pairing with steak'}, 'id': 'call_LrHyxTadqHDjW7J6LOWEaoSi', 'type': 'tool_call'}
tool_message = web_search.invoke(tool_call)
print(type(tool_message))

# 특정 속성들만 확인
print("\n=== 주요 속성들 ===")
attributes = ['content', 'tool_call_id', 'name', 'type', 'additional_kwargs']
for attr in attributes:
    if hasattr(tool_message, attr):
        print(f"{attr}: {getattr(tool_message, attr)}")


<class 'langchain_core.messages.tool.ToolMessage'>

=== 주요 속성들 ===
content: [{"title": "The Perfect Pairing: A Guide to Choosing the Best Wine for Your Steak ...", "url": "https://flavorycooking.com/what-wine-is-good-with-steak/", "content": "While red wine is often the traditional choice for pairing with steak, white wine can also be a good option. The key is to choose a white wine that’s full-bodied and rich enough to stand up to the flavor of the steak. Some good options for white wine pairings with steak include Chardonnay, Sauvignon Blanc, and Pinot Grigio. [...] The type of steak can have a significant impact on the wine pairing. Different types of steak have different flavor profiles, and some wines are better suited to certain types of steak than others. For example, a rich, fatty steak like a ribeye or a porterhouse pairs well with a full-bodied wine like a Cabernet Sauvignon or a Syrah/Shiraz. These wines have a rich, fruity flavor and a smooth, velvety texture that complemen

In [69]:
pprint(tool_message.content)


('[{"title": "The Perfect Pairing: A Guide to Choosing the Best Wine for Your '
 'Steak ...", "url": '
 '"https://flavorycooking.com/what-wine-is-good-with-steak/", "content": '
 '"While red wine is often the traditional choice for pairing with steak, '
 'white wine can also be a good option. The key is to choose a white wine '
 'that’s full-bodied and rich enough to stand up to the flavor of the steak. '
 'Some good options for white wine pairings with steak include Chardonnay, '
 'Sauvignon Blanc, and Pinot Grigio. [...] The type of steak can have a '
 'significant impact on the wine pairing. Different types of steak have '
 'different flavor profiles, and some wines are better suited to certain types '
 'of steak than others. For example, a rich, fatty steak like a ribeye or a '
 'porterhouse pairs well with a full-bodied wine like a Cabernet Sauvignon or '
 'a Syrah/Shiraz. These wines have a rich, fruity flavor and a smooth, velvety '
 'texture that complements the fatty flavor of

In [70]:
pprint(tool_message.tool_call_id)

'call_3LsmqtbHbUruyAHx625hicwl'


In [71]:
pprint(tool_message.name)

'tavily_search_results_json'


In [72]:
ai_msg.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'wines that pair well with steak'},
  'id': 'call_3LsmqtbHbUruyAHx625hicwl',
  'type': 'tool_call'}]

In [73]:
# batch 실행 - 도구 호출이 여러 개인 경우

# tool_messages = web_search.batch([tool_call])

tool_messages = web_search.batch(ai_msg.tool_calls)

print(tool_messages)
print("-" * 100)
pprint(tool_messages[0].content)

[ToolMessage(content='[{"title": "The Perfect Pairing: A Guide to Choosing the Best Wine for Your Steak ...", "url": "https://flavorycooking.com/what-wine-is-good-with-steak/", "content": "While red wine is often the traditional choice for pairing with steak, white wine can also be a good option. The key is to choose a white wine that’s full-bodied and rich enough to stand up to the flavor of the steak. Some good options for white wine pairings with steak include Chardonnay, Sauvignon Blanc, and Pinot Grigio. [...] The type of steak can have a significant impact on the wine pairing. Different types of steak have different flavor profiles, and some wines are better suited to certain types of steak than others. For example, a rich, fatty steak like a ribeye or a porterhouse pairs well with a full-bodied wine like a Cabernet Sauvignon or a Syrah/Shiraz. These wines have a rich, fruity flavor and a smooth, velvety texture that complements the fatty flavor of the steak. [...] When it comes 

`(4) ToolMessage를 LLM에 전달하여 답변을 생성하기`

* 파라미터 이름이 config여야 하는 이유: @chain 데코레이터가 config라는 이름만 특별히 인식하도록 설계되었습니다.
* config 생략 가능한 이유: @chain이 자동으로 기본 RunnableConfig()를 생성해서 전달하기 때문입니다.

In [75]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# Tavily 검색 도구 초기화 (최대 2개의 결과 반환)
tavily_tool = TavilySearchResults(max_results=12)

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_tool])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = tavily_tool.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
query = "스테이크와 어울리는 와인을 추천해주세요."
response = web_search_chain.invoke(query)

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_W74uH5dxBVUlJx2KszxvVtjC', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 109, 'total_tokens': 136, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BoQXZ0N79em4sV8o4A7tSZrBmTNeR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--4a25d0e4-3279-4300-9257-aaa4792f3a64-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_W74uH5dxBVUlJx2KszxvVtjC', 'type': 'tool_call'}] usage_metadata={'input_tokens': 109, 'output_tokens': 27,

### 2-2. 사용자 정의 도구
- @tool decorator를 통해 사용자 정의 도구를 정의할 수 있음

`(1) 도구(tool) 정의하기`

In [76]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

# Tool 정의 
@tool
def tavily_search_func(query: str) -> str:
    """Searches the internet for information that does not exist in the database or for the latest information."""

    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)

    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
        ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

### 2-3. Runnable 객체를 도구(tool) 변환
- 문자열이나 dict 입력을 받는 Runnable을 도구로 변환
- as_tool 메서드를 사용

- wikipedia 설치
```
    poetry add wikipedia
```
- [WikipediaLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.wikipedia.WikipediaLoader.html) 사용


In [ ]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하는 함수 
def search_wiki(input_data: dict) -> List[Document]:
    """Search Wikipedia documents based on user input (query) and return k documents"""
    query = input_data["query"]
    k = input_data.get("k", 2)  
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

# 도구 호출에 사용할 입력 스키마 정의 
class WikiSearchSchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documents to return (default is 2)")

# RunnableLambda 함수를 사용하여 위키피디아 문서 로더를 Runnable로 변환 
runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a specified number of documents. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSearchSchema)

## tool 정의 방법은 2가지가 존재
## as_tool을 사용하는 방법
## tavily_search_func 같이 적용하는 방법

In [80]:
# 도구 속성
print("자료형: ")
print(type(wiki_search))
print("-"*100)

print("name: ")
print(wiki_search.name)
print("-"*100)

print("description: ")
pprint(wiki_search.description)
print("-"*100)

print("schema: ")
pprint(wiki_search.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_search
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a specified number of documents. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'k': {'default': 2,
                      'description': 'The number of documents to return '
                                     '(default is 2)',
                      'title': 'K',
                      'type': 'integer'},
                'q

In [81]:
# 위키 검색 실행
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query":query})

# 검색 결과 출력
for result in wiki_results:
    print(result)  
    print("-" * 100)  

page_content='오르조(이탈리아어: orzo) 또는 리소니(이탈리아어: risoni, 단수: risone 리소네[*])는 이탈리아의 파스타이다. "오르조"는 라틴어:hordeum에서 유래했으며 "보리"를 뜻한다. "리소니"는 "큰 쌀"이라는 뜻이다. 파스타의 일종으로 큰 쌀알의 모양을 하고 있으며 솔방울이나 잣보다는 좀 더 작다. 보통 라구 등 수프와 함께 먹는다. 원래는 보리로 만들었지만 요즘에는 박력분으로 만드는 것이 흔해졌다. 다른 이름으로는 kritharáki ("little barley") 혹은 manéstra(그리스 요리)로 부르며, lisān al-`uṣfūr ("명금의 혀")라고 아랍 요리에서 불린다. 오르조를 두고 이탈리아의 쌀이라고 부르기도 한다.
터키에서는 아르파 셰흐리예(arpa şehriye)로 불리며, 셰흐리예의 하나이다. 필라브를 만들거나 초르바(수프)에 넣어 먹는다.


== 같이 보기 ==
파스티나
셰흐리예


== 각주 ==' metadata={'title': '오르조', 'summary': '오르조(이탈리아어: orzo) 또는 리소니(이탈리아어: risoni, 단수: risone 리소네[*])는 이탈리아의 파스타이다. "오르조"는 라틴어:hordeum에서 유래했으며 "보리"를 뜻한다. "리소니"는 "큰 쌀"이라는 뜻이다. 파스타의 일종으로 큰 쌀알의 모양을 하고 있으며 솔방울이나 잣보다는 좀 더 작다. 보통 라구 등 수프와 함께 먹는다. 원래는 보리로 만들었지만 요즘에는 박력분으로 만드는 것이 흔해졌다. 다른 이름으로는 kritharáki ("little barley") 혹은 manéstra(그리스 요리)로 부르며, lisān al-`uṣfūr ("명금의 혀")라고 아랍 요리에서 불린다. 오르조를 두고 이탈리아의 쌀이라고 부르기도 한다.\n터키에서는 아르파 셰흐리예(arpa şehriye)로 불리며, 셰흐리예의 하나이다. 필라브를 만들거나 초르바(수프)에 넣어 먹는다.', 'source': 'https://ko.wik

In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_search])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

## 웹 검색, 특정 위키피디아 사이드라 던가 다양한 도구들을 결합해서 사용해야 원하는 정확한 답변을 들을 수 있음
## 진단 분야의 정확도를 높이기 위해서는 벡터 디비를 활용해서 다양한 치료 논문들의 치료법들을 vector db의 형태로 가지고 있어야
## 보다 정확한 진단 초안을 작성이 가능할 것..


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ojyrAOpugHtoYeH6BVS0mZTK', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_nUpSNfPvGRdEuMGE7s0ir3FT', 'function': {'arguments': '{"query": "파스타 유래"}', 'name': 'wiki_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 176, 'total_tokens': 233, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BoQleFpSDiUEDLvLnPDrn2gWAduB9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--94fda710-9364-4280-8d37-e8060c7e93ac-0', tool_calls=[{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, 'id

`(2) LCEL 체인`
- 위키피디아 문서를 검색하고 내용을 요약하는 체인

In [84]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader

# WikipediaLoader를 사용하여 위키피디아 문서를 검색하고 텍스트로 반환하는 함수 
def wiki_search_and_summarize(input_data: dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs =[
        f'<Document source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
        ]
    
    return formatted_docs

# 요약 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

# LLM 및 요약 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context": RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser() 
)

# 요약 테스트 
summarized_text = summary_chain.invoke({"query":"파스타의 유래"})
pprint(summarized_text)

('오르조(또는 리소니)는 이탈리아의 파스타로, 보리에서 유래한 이름을 가지고 있으며 큰 쌀알 모양이다. 주로 수프와 함께 먹으며, 현재는 '
 '박력분으로 만들어진다. 터키에서는 아르파 셰흐리예로 불리며, 필라브나 수프에 사용된다. \n'
 '\n'
 '이탈리아 요리는 기원전 4세기부터 발전해온 남유럽의 요리로, 지역마다 특색이 다르다. 북부는 쌀과 유제품을, 남부는 올리브와 해산물을 '
 '주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, 2013년 CNN에서 세계 최고의 요리로 선정되었다. 요리의 역사적 '
 '발전은 고대 그리스와 로마, 중세 아랍 문화의 영향을 받았으며, 다양한 요리책이 출판되었다.')


In [85]:
# 도구 호출에 사용할 입력 스키마 정의 
class WikiSummarySchema(BaseModel):
    """Input schema for Wikipedia search."""
    query: str = Field(..., description="The query to search for in Wikipedia")

# as_tool 메소드를 사용하여 도구 객체로 변환
wiki_summary = summary_chain.as_tool(
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

# 도구 속성
print("자료형: ")
print(type(wiki_summary))
print("-"*100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Input schema for Wikipedia search.',
 'properties': {'query': {'description': 'The query to search for in Wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------

In [86]:
# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[tavily_search_func, wiki_summary])

# 도구 호출이 필요한 LLM 호출을 수행
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5Y9SCQWP3XO2f4NsY7FqlbJz', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'tavily_search_func'}, 'type': 'function'}, {'id': 'call_Lk9o5g3OJVLucINm0bGqTi1h', 'function': {'arguments': '{"query": "파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 152, 'total_tokens': 210, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BoQrnFMlV7qywAzCf53vYUinOppcB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--63a577e2-41ac-4b66-84c0-74b89695fb10-0', tool_calls=[{'name': 'tavily_search_func', 'args': {'query': '서울 강남 파스타 맛집'}, '

In [87]:
ai_msg.tool_calls[1]

{'name': 'wiki_summary',
 'args': {'query': '파스타의 유래'},
 'id': 'call_Lk9o5g3OJVLucINm0bGqTi1h',
 'type': 'tool_call'}

In [88]:
# 도구 실행 
tool_message = wiki_summary.invoke(ai_msg.tool_calls[1])

print(tool_message)
print("-" * 100)
pprint(tool_message.content)

content='오르조(또는 리소니)는 이탈리아의 파스타로, 보리에서 유래한 이름을 가지고 있으며 큰 쌀알 모양이다. 주로 수프와 함께 먹으며, 현재는 박력분으로 만들어진다. 터키에서는 아르파 셰흐리예로 불리며, 필라브나 수프에 사용된다. \n\n이탈리아 요리는 기원전 4세기부터 발전해온 남유럽의 요리로, 지역마다 특색이 다르다. 북부는 쌀과 유제품을, 남부는 올리브와 해산물을 주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, 2013년 CNN에서 세계 최고의 요리로 선정되었다. 요리의 역사적 발전은 고대 그리스와 로마, 중세 아랍 문화의 영향을 받았으며, 다양한 요리책이 출판되었다.' name='wiki_summary' tool_call_id='call_Lk9o5g3OJVLucINm0bGqTi1h'
----------------------------------------------------------------------------------------------------
('오르조(또는 리소니)는 이탈리아의 파스타로, 보리에서 유래한 이름을 가지고 있으며 큰 쌀알 모양이다. 주로 수프와 함께 먹으며, 현재는 '
 '박력분으로 만들어진다. 터키에서는 아르파 셰흐리예로 불리며, 필라브나 수프에 사용된다. \n'
 '\n'
 '이탈리아 요리는 기원전 4세기부터 발전해온 남유럽의 요리로, 지역마다 특색이 다르다. 북부는 쌀과 유제품을, 남부는 올리브와 해산물을 '
 '주로 사용한다. 이탈리아 요리는 간결함과 재료의 질을 중시하며, 2013년 CNN에서 세계 최고의 요리로 선정되었다. 요리의 역사적 '
 '발전은 고대 그리스와 로마, 중세 아랍 문화의 영향을 받았으며, 다양한 요리책이 출판되었다.')


In [89]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM에 도구를 바인딩
llm_with_tools = llm.bind_tools(tools=[wiki_summary])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def wiki_summary_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-"*100)

    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")

# 응답 출력 
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_wpnSUik9Ws8UzpwNU36ROiFA', 'function': {'arguments': '{"query":"파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 120, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BoQrxAy3af69AJM4dhand00X8V6Gg', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--64a1c52e-4220-4ae1-a4c0-b0cc6d848a42-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_wpnSUik9Ws8UzpwNU36ROiFA', 'type': 'tool_call'}] usage_metadata={'input_tokens': 120, 'output_tokens': 19, 'total_tokens': 139, 'input_token_details': {

### 2-4. 벡터저장소 검색기
- @tool decorator 사용

`(1) 문서 로드 및 인덱싱`

In [90]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [91]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크
내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, ...

메뉴 번호: 2
메뉴 이름: 트러플 리조또
내용:
2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조...


### Embedding을 하기 위해 Ollama Qwen2.5 설치
```
ollama run qwen2.5:1.5b

ollama ls
```

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="qwen2.5:1.5b") 

# FAISS 인덱스 생성
menu_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# FAISS 인덱스 저장 (선택사항)
menu_db.save_local("./db/menu_db")


# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

- 와인 메뉴에 대해서도 같은 작업을 처리

In [ ]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# FAISS 인덱스 생성
wine_db = FAISS.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model
)

# 병합된 DB 다시 저장
wine_db.save_local("./db/wine_db")

wine_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

`(2) 도구(tool) 정의하기`

In [ ]:
from langchain_core.tools import tool
# menu db 벡터 저장소 로드
menu_db = FAISS.load_local(
    "./db/menu_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_menu_func(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_menu_func))
print("-"*100)

print("name: ")
print(db_search_menu_func.name)
print("-"*100)

print("description: ")
pprint(db_search_menu_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_menu_func.args_schema.schema())
print("-"*100)

In [ ]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# wine db 벡터 저장소 로드
wine_db = FAISS.load_local(
    "./db/wine_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

@tool
def db_search_wine_func(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(db_search_wine_func))
print("-"*100)

print("name: ")
print(db_search_wine_func.name)
print("-"*100)

print("description: ")
pprint(db_search_wine_func.description)
print("-"*100)

print("schema: ")
pprint(db_search_wine_func.args_schema.schema())
print("-"*100)

In [ ]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[db_search_menu_func, db_search_wine_func])

# 도구 호출이 필요한 LLM 호출을 수행
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 와인도 추천 해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

`(3) 여러 개의 도구(tool) 호출하기`

In [ ]:
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
for tool in tools:
    print(tool.name)

In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 와인도 해주세요.")

# 응답 출력 
print(response.content)

In [ ]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 파스타의 역사 또는 유래를 알려주세요.")

# 응답 출력 
print(response.content)

## 3. Few-shot 프롬프팅 
- 각 도구의 용도를 구분하여 few-shot 예제로 제시

### 3-1. Few-shot 도구 호출

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_menu_func", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),    
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. \
                주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "db_search_wine_func", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. \
                2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다.\
               주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, \
              주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. \
              특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오,\
               그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인도 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

In [ ]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

### 3-2. 답변 생성 체인 

In [ ]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        # [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
        if tool_call["name"] == "tavily_search_func":
            tool_message = tavily_search_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_wine_func":
            tool_message = db_search_wine_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "db_search_menu_func":
            tool_message = db_search_menu_func.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해 주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

In [ ]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

# 응답 출력 
pprint(response.content)

## 4. LangChain Agent 사용
- 유의사항: 프롬프트에 "agent_scratchpad",  "input" 변수를 포함

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
        2. For general food information, history, and cultural background, utilize the wiki_summary tool.
        3. For wine recommendations or food and wine pairing information, use the search_wine tool.
        4. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
        5. Provide clear and concise responses based on the search results.
        6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        7. Always maintain a helpful and professional tone.
        8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [ ]:
# Tool calling Agent 생성
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [tavily_search_func, wiki_summary, db_search_menu_func, db_search_wine_func]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# AgentExecutor 생성 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# AgentExecutor 실행

query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
agent_response = agent_executor.invoke({"input": query})

In [ ]:
pprint(agent_response)

## 5. Gradio 활용

In [ ]:
import gradio as gr
from typing import List, Tuple

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        # 채팅 기록을 AI에게 전달할 수 있는 형식으로 변환
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
        
        # agent_executor를 사용하여 응답 생성
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history[-2:]    # 최근 2개의 메시지 기록만을 활용 
        })
        
        # agent_executor의 응답에서 최종 답변 추출
        return response['output']
    except Exception as e:
        # 오류 발생 시 사용자에게 알리고 로그 기록
        print(f"Error occurred: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."

# 예제 질문 정의
example_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

# Gradio 인터페이스 생성
demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다.",
    examples=example_questions,
    theme=gr.themes.Soft()
)

# 데모 실행
demo.launch()

In [ ]:
# 데모 종료
demo.close()